<a href="https://colab.research.google.com/github/sudislife/Google-Stock-Price-Prediction-Using-Simple-RNN/blob/main/Google_Stock_Price_Prediction_Using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the data

In [ ]:
import pandas as pd

google_stock_train = pd.read_csv('https://raw.githubusercontent.com/sudislife/DatasetsForColab/main/Google_Stock_Price_Train.csv')
google_stock_test = pd.read_csv('https://raw.githubusercontent.com/sudislife/DatasetsForColab/main/Google_Stock_Price_Test.csv')

In [ ]:
google_stock_train = pd.concat((google_stock_train, google_stock_test.loc[pd.DatetimeIndex(google_stock_test['Date']).month < 6]), axis=0)
google_stock_test = google_stock_test.loc[pd.DatetimeIndex(google_stock_test['Date']).month >= 6]

In [ ]:
google_stock_train['Date'] = pd.to_datetime(google_stock_train['Date'])
google_stock_test['Date'] = pd.to_datetime(google_stock_test['Date'])

In [ ]:
print('Shape of google stock train:', google_stock_train.shape)
print('Shape of google stock test:', google_stock_test.shape)

Shape of google stock train: (4223, 8)
Shape of google stock test: (33, 8)


In [ ]:
google_stock_train.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume
0,0,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942874
1,1,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342897
2,2,2004-08-24,55.412300,55.591629,51.591621,52.239197,52.239197,15319808
3,3,2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232276
4,4,2004-08-26,52.279045,53.773445,52.134586,53.753517,53.753517,7128620


In [ ]:
google_stock_test.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume
102,4223,2021-06-01,2422.000000,2437.970947,2404.879883,2429.810059,2429.810059,958300
103,4224,2021-06-02,2435.310059,2442.000000,2404.199951,2421.280029,2421.280029,857900
104,4225,2021-06-03,2395.020020,2409.745117,2382.830078,2404.610107,2404.610107,917300
105,4226,2021-06-04,2422.520020,2453.858887,2417.770020,2451.760010,2451.760010,1297400
106,4227,2021-06-07,2451.320068,2468.000000,2441.072998,2466.090088,2466.090088,1192500


In [ ]:
google_stock_train['Date'].max()

Timestamp('2021-05-28 00:00:00')

In [ ]:
google_stock_test['Date'].max()

Timestamp('2021-07-16 00:00:00')

## Visualizing the data

In [ ]:
google_stock_train.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
import plotly.express as px

fig = px.line(
    google_stock_train, 
    x='Date', 
    y='Open', 
    title='Open vs Date Plot for Google Stock Price',
    template='plotly_dark'
)
fig.show()

In [ ]:
import plotly.express as px

fig = px.line(
    google_stock_test, 
    x='Date', 
    y='Open', 
    title='Open vs Date Plot for Google Stock Price (on Test Data)',
    template='plotly_dark'
)
fig.show()

## Feature Scaling 

In [ ]:
training_set = google_stock_train['Open'].values.reshape(-1, 1)
training_set

array([[  50.316402],
       [  55.168217],
       [  55.4123  ],
       ...,
       [2412.834961],
       [2436.939941],
       [2421.959961]])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Using Normalization over here because Standardization doesn't work here (find out why later)
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)
training_set_scaled

array([[3.79723374e-04],
       [2.41187154e-03],
       [2.51410397e-03],
       ...,
       [9.89903801e-01],
       [1.00000000e+00],
       [9.93725742e-01]])

In [ ]:
X_train = []
y_train = []
for i in range(60, 4223):
  X_train.append(training_set_scaled[i-60:i, 0])
  y_train.append(training_set_scaled[i, 0])

In [ ]:
import numpy as np

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_train.shape

(4163, 60)

In [ ]:
X_train = np.reshape(X_train, (4163, 60, 1))

In [ ]:
X_train.shape

(4061, 60, 1)

## Building the RNN

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [ ]:
LSTM_Model = Sequential()

# Adding the first LSTM layer and some Dropout regularization
LSTM_Model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
LSTM_Model.add(Dropout(0.2))
LSTM_Model.add(LSTM(50, return_sequences=True))
LSTM_Model.add(Dropout(0.2))
LSTM_Model.add(LSTM(50, return_sequences=True))
LSTM_Model.add(Dropout(0.2))
LSTM_Model.add(LSTM(50))
LSTM_Model.add(Dropout(0.2))
LSTM_Model.add(Dense(1))
LSTM_Model.compile(optimizer='adam', loss='mean_squared_error')
model = None
model = LSTM_Model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
131/131 [==============================] - 13s 15ms/step - loss: 0.0048
Epoch 2/100
131/131 [==============================] - 2s 14ms/step - loss: 0.0012
Epoch 3/100
131/131 [==============================] - 2s 13ms/step - loss: 0.0011
Epoch 4/100
131/131 [==============================] - 2s 13ms/step - loss: 9.3841e-04
Epoch 5/100
131/131 [==============================] - 2s 13ms/step - loss: 0.0010
Epoch 6/100
131/131 [==============================] - 2s 13ms/step - loss: 8.9273e-04
Epoch 7/100
131/131 [==============================] - 2s 13ms/step - loss: 8.7510e-04
Epoch 8/100
131/131 [==============================] - 2s 14ms/step - loss: 8.5133e-04
Epoch 9/100
131/131 [==============================] - 2s 14ms/step - loss: 7.5766e-04
Epoch 10/100
131/131 [==============================] - 2s 14ms/step - loss: 7.2579e-04
Epoch 11/100
131/131 [==============================] - 2s 13ms/step - loss: 8.5551e-04
Epoch 12/100
131/131 [==============================] - 

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = np.arange(5, 100), y = model.history['loss'][5:], name = 'Loss in training data'))
fig.update_layout(title = 'Loss Graph of RNN', template='plotly_dark')
fig.update_xaxes(title_text='Epochs')
fig.update_yaxes(title_text='Loss')
fig.show()

In [ ]:
LSTM_Model.save('LSTM_Model_of_Google')

INFO:tensorflow:Assets written to: LSTM_Model_of_Google/assets


INFO:tensorflow:Assets written to: LSTM_Model_of_Google/assets


## Making the predictions and visualizing the results

In [ ]:
total_set = pd.concat((google_stock_train['Open'], google_stock_test['Open']), axis=0)
inputs = total_set[total_set.shape[0] - google_stock_test.shape[0] - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

In [ ]:
google_stock_test.shape

(33, 8)

In [ ]:
X_test = []
y_test = []
for i in range(60, 93):
  X_test.append(inputs[i-60:i, 0])
  y_test.append(inputs[i, 0])

In [ ]:
import numpy as np

X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_test.shape

(33, 60)

In [ ]:
X_test = np.reshape(X_test, (X_test.shape[0], 60, 1))

In [ ]:
predicted_test_stock = LSTM_Model.predict(X_test)
predicted_test_stock = sc.inverse_transform(predicted_test_stock)

In [ ]:
pred_test = np.reshape(predicted_test_stock, (predicted_test_stock.shape[0]))

In [ ]:
google_stock_test['Pred'] = pred_test

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = google_stock_test['Date'], y = google_stock_test['Open'], name = 'Open in testing data'))
fig.add_trace(go.Scatter(x = google_stock_test['Date'], y = google_stock_test['Pred'], name = 'Pred in testing data'))
fig.update_layout(title = 'Real vs Pred Stock Price ', template='plotly_dark')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Open')
fig.show()

## Evaulating the RNN

In [ ]:
import math
from sklearn.metrics import mean_squared_error
rmse = math.sqrt(mean_squared_error(google_stock_test['Open'], google_stock_test['Pred']))

In [ ]:
print('Average of Open', total_set.mean())

Average of Open 603.4385776489655


In [ ]:
print('RMSE: ', rmse)

RMSE:  131.55579516150573
